In [ ]:
from os import walk
import os
import re
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
%matplotlib inline
import seaborn as sns
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D


sns.reset_orig()
sns.set_style("darkgrid")

remote_paths=[]
remote_paths.append("")
remote_paths.append("/usr/scratch2/larain8/cykoenig/development/mempool_dphpc/jupyter/")

#path to your input result folder

#path to the output csv
csv_path= "./argmax_results_3.csv"


def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True   # Jupyter notebook or qtconsole
        elif shell == 'TerminalInteractiveShell':
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False      # Probably standard Python interpreter



In [ ]:
cols_to_keep = ['num_cores', 'num_data', 'core', 'section', 'seed', 'max_val', 'cs_retry', 'cs_duration', 'cycles', 'algo']

tot_df = pd.DataFrame()

for remote_path in remote_paths:
    
    result_path = remote_path+"../hardware/results"
    print("\nProcessing files at", result_path)

    # Add the lock informations in the results.csv files
    !python ../onnx_benchmark/analyze_locks.py VERBOSE=0 REMOTE_PATH={remote_path}

    todo, done = 0, 0
    for (dirpath, dirnames, filenames) in walk(result_path):
        for filename in filenames:
            if filename == "transcript":
                todo+=1

    configs = []
    for (dirpath, dirnames, filenames) in walk(result_path):
        num_cores = -1
        num_data = -1
        num_local_data = -1
        max_val = -1
        seed = -1
        to_find = {"lobal max":[], "ndexes len":[]}
        stop = False
            
        for filename in filenames:
            if filename != "transcript":
                continue
            p = re.compile("[0-9]+")
            # Get parameters from the directory name first
            num_cores, num_data, max_val, seed = dirpath.split('/')[-2].split('_')[-4:]
            algo = "_".join(dirpath.split('/')[-2].split('_')[:-4])
            num_cores, num_data, max_val, seed = int(num_cores), int(num_data), int(max_val), int(seed)
            nul_local_data = int(num_data / num_cores)
        
            # Override them based on transcript if any
            with open(dirpath+"/"+filename, "r") as fp:

                for line in fp.readlines():
                    found = False
                    for word in to_find.keys():
                        if word in line and not found:
                            to_find[word].append([a.group() for a in re.compile("[0-9]+").finditer(line)][0])
                            found = True
                
                    if "ERROR" in line:
                        print("ERROR in ", dirpath)
                        assert(0)
                assert(to_find["lobal max"][0]==to_find["lobal max"][1] and to_find["ndexes len"][0]==to_find["ndexes len"][1])

            # Read CSV
            done += 1
            df = pd.read_csv(dirpath+"/results.csv")
            df["num_cores"] = int(num_cores)
            df["num_data"] = int(num_data)
            df["num_local_data"] = int(num_local_data)
            df["max_val"] = int(max_val)
            df["seed"] = int(seed)
            df["algo"] = algo
            
            # Delete column without name
            df = df.reset_index(drop=True)
            for col in df.columns:
                if not col in cols_to_keep:
                    df = df.drop(col, 1)
            # Print state
            print("\r{}/{}".format(done, todo) + "    cores={0:3d}".format(int(num_cores))+"    data={0:7d}b".format(int(num_data))\
                  +"    max_gen={0:7d}".format(int(max_val))+"    max={0:7d}".format(int(to_find['lobal max'][0]))+"    len={0:7d}".format(int(to_find['ndexes len'][0])), end="")
            # Concat everyone
            configs.append([num_cores, num_data, num_local_data])
            tot_df = pd.concat([tot_df, df], axis=0, join="outer", ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False, copy=True)
            if(tot_df['cs_retry'].isnull().any()):
                print("\r"+dirpath+"/"+filename+" uncorrect cs_retry, rerun analyze_locks")
            if(tot_df['core'].isnull().any()):
                print("\r"+dirpath+"/"+filename+" uncorrect core")
        
#raw_df = tot_df.copy()
# Drop nans
to_drop = tot_df.columns[tot_df.isnull().any()]
print("\nDropping :")
for nan_col in to_drop:
    assert(not nan_col in ['cycles', 'num_data', 'num_cores'])
    tot_df = tot_df.drop(nan_col, axis=1)
    print(nan_col, end=" ")

# Drop strings
tot_df = tot_df #.select_dtypes(exclude=['object'])

tot_df.to_csv(csv_path)
print("\nResults exported to", csv_path)

print(tot_df.columns)


In [ ]:
tot_df = pd.read_csv(csv_path)
print(tot_df.shape)
print(tot_df.head())

In [ ]:
# iddx of the CI of the median with 95%
def ci_idx(n):
    return [int((n-1.96*np.sqrt(n))/2), int(np.ceil(1+(n+1.96*np.sqrt(n))/2))]

# return (ci_low, median, ci_high) for a list of values
def median_and_ci(data):
    n = len(data)
    a, b = ci_idx(n)
    data_sorted = sorted(data)
    return data_sorted[a], data_sorted[int(n/2)], data_sorted[min(b, n-1)]

def get_median(data):
    return median_and_ci(data)[1]
def get_ci(data):
    a, b, c = median_and_ci(data)
    return a, c

def get_ideal_speedup(a, b, n=100):
    X = np.linspace(a, b, num=n)
    return X, X

In [ ]:
print("Tot_df :", tot_df.shape)

colors = [(4, "green"), (64, "green"), (200, "green")]


#
# Strong scaling with confidence intervals
#

print(tot_df['num_data'].unique())
print(tot_df['num_cores'].unique())


algo = "argmax_impl"
for num_data, color in colors: #tot_df['num_data'].unique():
    experiment_df = tot_df[(tot_df['num_data'] == num_data) & ((tot_df['algo'] == "argmax_base")|(tot_df['algo'] == algo))]
    print("Experiment :", experiment_df.shape)
    Y1, Y1_low, Y1_high = [], [], []
    Y1_fill, Y1_argmax, Y1_reduction = [], [], []
    num_cores_arr = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    all_cores = []
    all_tot = []
    for num_cores in num_cores_arr:
        res_fill = []
        res_argmax = []
        res_reduction = []
        res_tot = []
        maxs = []
        for seed in experiment_df['seed'].unique():
            mask = (experiment_df['seed'] == seed) & (experiment_df['num_cores'] == num_cores)
            masked = experiment_df[mask]
            if masked.empty:
                continue
            res_fill.append(      masked.loc[masked['section'] == 0]['cycles'].max() / 1000000 )
            res_argmax.append(    masked.loc[masked['section'] == 1]['cycles'].max() / 1000000 )
            res_reduction.append( masked.loc[masked['section'] == 2]['cycles'].max() / 1000000 )
            # All does not count fill section
            res_tot.append(       masked[masked['section']==1].groupby(['core']).cycles.sum().max() / 1000000)
            all_cores.append(     num_cores)
            all_tot.append(       masked[masked['section']!=0].groupby(['core']).cycles.sum().max() / 1000000)
        if len(res_tot) == 0:
            continue
        lo, median, hi = median_and_ci(res_tot)
        Y1.append(median)
        Y1_low.append(lo)
        Y1_high.append(hi)
        Y1_fill.append(get_median(res_fill))
        Y1_argmax.append(get_median(res_argmax))
        Y1_reduction.append(get_median(res_reduction))
        print("(", num_cores, ",", len(res_tot), ")", end=" : ")
        print("(", lo, ",", median, ",", hi, ")")
    
    Y1 = np.array(Y1)
    all_cores = np.array(all_cores)
    all_tot = np.array(all_tot)
    print(all_cores)
    
    #plot_errorbars()
    fig = plt.figure(figsize=(8, 6), dpi=80)
    ax = fig.add_subplot(1, 1, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_xscale('log', basex=2)
    ax.set_xlim(1, 256)
    ax.set_xticks(num_cores_arr)
    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    #sns.pointplot(num_cores_arr, Y1[0]/Y1, errorbar=get_ci, join=False, ax=ax)
    #sns.pointplot(num_cores_arr, 2*Y1[0]/Y1, errorbar=get_ci, join=False, ax=ax)
    plt.plot(num_cores_arr, Y1[0]/Y1, '-o', label="Observed")
    plt.plot(get_ideal_speedup(1, 256)[0], get_ideal_speedup(1, 256)[1], '--', color="red", label="Ideal")
    plt.legend()
    plt.title("Argmax strong scaling with " + str(num_data * 256) + " datas (int32)")
    plt.xlabel("# cores")
    plt.ylabel("Speedup")
    
    print(Y1[0]/Y1)
    
    fig = plt.figure(figsize=(8, 6), dpi=80)
    ax = fig.add_subplot(1, 1, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plot_df = pd.DataFrame({'Cores': num_cores_arr, 'Fill': Y1_fill, 'Argmax': Y1_argmax, 'Reduction': Y1_reduction}, \
                           columns = ['Cores', 'Fill', 'Argmax', 'Reduction'])
    plot_df = plot_df.set_index('Cores')
    plot_df.drop(columns=['Fill']).plot(kind='bar', stacked=True, color=['orange', 'blue'], ax=ax)
    plt.title("Computing time distribution with " + str(num_data * 256) + " datas (int32)")
    plt.xlabel("# cores")
    plt.ylabel("Time (Mcycles)")
    
    fig = plt.figure(figsize=(8, 6), dpi=80)
    filter_box = all_cores == 1
    
    # Remove median
    for num_cores in num_cores_arr:
        idx = np.where(all_cores == num_cores)
        median = get_median(all_tot[idx])
        #all_tot[idx] -= median

    ax = fig.add_subplot(1, 1, 1)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    plot_df = pd.DataFrame({'Cores': all_cores,
                            'Total': all_tot}, \
                           columns = ['Cores', 'Total'])
    sns.boxplot(data=plot_df, x="Cores", y="Total", whis=10, ax=ax)
    plt.title("Variability with " + str(num_data * 256) + " datas (int32)")
    plt.xlabel("# cores")
    plt.ylabel("Time (Mcycles)")
    

In [ ]:
print("Tot_df :", tot_df.shape)

colors = [(1, "Serial"), (256, "Parallel")]
algos = [('argmax_impl_2', 'lightgreen'), ('argmax_impl', 'lightblue')]

legend_elements = []
for a, b in algos:
    legend_elements+=[Line2D([0], [0], color=b, lw=4, label=a)]

#plot_errorbars()
fig = plt.figure(0, figsize=(8, 6), dpi=80)
ax = fig.add_subplot(1, 1, 1)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xscale('log', basex=2)
ax.set_xlim(1, 256)
ax.set_xticks(sorted(tot_df['num_data'].unique()))
ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
plt.title("Speedup with input size")
plt.xlabel("Input size (KiB)")
plt.ylabel("Time (Mcycles)")

#
# Strong scaling with confidence intervals
#


Y1_0 = []
for num_cores, label in colors: #tot_df['num_data'].unique():
    for algo, algo_color in algos:
        experiment_df = tot_df[(tot_df['num_cores'] == num_cores) & ((tot_df['algo'] == "argmax_base")|(tot_df['algo'] == algo))]
        print("Num cores:", num_cores,"Experiment :", experiment_df.shape)
        Y1, Y1_low, Y1_high = [], [], []
        Y1_fill, Y1_argmax, Y1_reduction = [], [], []
        num_data_arr = sorted(tot_df['num_data'].unique())
        all_data = []
        all_tot = []
    
        mask = (experiment_df['num_data'] == 200)
    
        for num_data in num_data_arr:
            res_fill = []
            res_argmax = []
            res_reduction = []
            res_tot = []
            for seed in experiment_df['seed'].unique():
                mask = (experiment_df['seed'] == seed) & (experiment_df['num_data'] == num_data)
                masked = experiment_df[mask]
                if masked.empty:
                    continue
                res_fill.append(      masked.loc[masked['section'] == 0]['cycles'].max() / 1000000 )
                res_argmax.append(    masked.loc[masked['section'] == 1]['cycles'].max() / 1000000 )
                res_reduction.append( masked.loc[masked['section'] == 2]['cycles'].max() / 1000000 )
                # All does not count fill section
                res_tot.append(       masked[masked['section']!=0].groupby(['core']).cycles.sum().max() / 1000000)
                all_data.append(      num_data)
                all_tot.append(       masked[masked['section']!=0].groupby(['core']).cycles.sum().max() / 1000000)
            if len(res_tot) == 0:
                continue
            lo, median, hi = median_and_ci(res_tot)
            Y1.append(median)
            Y1_low.append(lo)
            Y1_high.append(hi)
            Y1_fill.append(get_median(res_fill))
            Y1_argmax.append(get_median(res_argmax))
            Y1_reduction.append(get_median(res_reduction))
            print("(", num_data, ",", len(res_tot), ")", end=" : ")
            print("(", lo, ",", median, ",", hi, ")")
    
        Y1 = np.array(Y1)
        all_tot = np.array(all_tot)
        fig = plt.figure(0, figsize=(8, 6), dpi=80)
        plt.plot(num_data_arr, Y1, "-o", label=label)
        
        if num_cores != 1:
            #plt.plot(num_data_arr, Y1_0/Y1, "-o", label=label)
            pass
    
        if num_cores == 1:
            Y1_0 = Y1
    
        new_fig = plt.figure(num_cores, figsize=(8, 6), dpi=80)
        # Remove median
        for num_data in num_data_arr:
            idx = np.where(all_data == num_data)
            #median = get_median(all_tot[idx])
            #all_tot[idx] -= median

        ax2 = new_fig.add_subplot(1, 1, 1)
        ax2.spines['right'].set_visible(False)
        ax2.spines['top'].set_visible(False)
        plot_df = pd.DataFrame({'Datas': all_data,
                                'Total': all_tot}, \
                               columns = ['Datas', 'Total'])
        sns.boxplot(data=plot_df, x="Datas", y="Total", whis=10, ax=ax2, color=algo_color)
        plt.title("Execution time with input size for "+str(num_cores)+" cores")
        plt.xlabel("Input size (KiB)")
        plt.ylabel("Time (Mcycles)")
        plt.legend(handles=legend_elements)


In [ ]:
grouped = tot_df.groupby(["num_cores", "num_data", "section"]).agg({'cs_retry': 'sum', 'cs_duration': 'sum'})

In [ ]:
num_cores = 256
num_data = 200
section = 1

for num_data in [8, 32, 64, 200]:
    plt.figure()
    strin = grouped.loc[num_cores,num_data,section]['cs_retry'].replace('][',',').replace('[', '').replace(']', '').replace(' ','')
    distrib = [int(x) for x in strin.split(',') if x != '']
    plt.hist(distrib, bins=20, density=True)
print("Done")